In [1]:
# perform the same type of analysis performed on adults for 4D time-series tif files
# iterate over folders that has the image file, and a mask file
# the subtype has to be provided (for now)
# Also reads a txt file for all entry_points

For explanation of the the individual columns in the result csv file, expand the next cell

In [2]:
"""" Results' columns' description 
'neuron':  neuron name extracted from file name
'subtype': neuron name extracted from file name
'entry_point': entry point of the neuron, read from seperate txt file

'volume': volume of the masked 3D image in um3
'3D_convex_volume': volume of the convex-hull for the 3D image in um3
'net_area': area of the object in the 2D image (the 2D is a max projection of the 3D image) in um2
'area': area of the convex_hull surronding the 2D image (obtained from skimage.measure.regionprops) in um2 

'density_2D': solidity value obtained from skimage.measure.regionprops on the 2D image
'density_3D': pixel count of 3D image/pixel count of 3D convexhull 
'object_perimeter': object perimenter value obtained from skimage.measure.regionprops on the 2D image
'convex_perimeter': perimeter of the surronding convex_hull

'AP_spread': (max-min) of value pixels index in last (x) dimension of the 2D image in um
'DV_spread': (max-min) of value pixels index in (y) dimension of the 2D image in um                
'ellipticity': (elipse_major - elipse_minor)/ elipse_major
'PCA_elongation': (max_eign_value - min_eign_value)/ max_eign_value
'eccentricity': eccentricity value obtained from skimage.measure.regionprops on the 2D image 
'circularity': (4*np.pi*convex_area)/(convex_perimeter**2)


'compactness': object_perimeter**2/(4*np.pi*net_area)
'surface_area': mesh_surface_area of the marching_cubes on the 3D masked image in um2
'3D_convex_surface': mesh_surface_area of the marching_cubes on the 3D masked image in um2



'convexity_2D': convex_perimeter/object_perimeter
'convexity_3D': 3D_convex_surface/surface_area
'orientation': orientation value obtained from skimage.measure.regionprops on the 2D image 
'feret_diameter_max': feret_diameter_max value obtained from skimage.measure.regionprops on the 2D image 

'centroidZ': Z position of the centroid from regionprops on the 3D masked image 
'centroidY': Y position of the centroid from regionprops on the 3D masked image 
'centroidX': X position of the centroid from regionprops on the 3D masked image 
'cen_EP_dis': ecludian distance of the centroid from the entry point
'cen_EP_angle': angle of the line between centroid from the entry point (np.arctan2)
'cen_EP_y', 'cen_EP_x': the x,y position of the centroid with the entry point as center (0,0) 

£££ PCA analysis
'asymmetry', 'x_asymmetry', 'y_asymmetry', 'PC1_asymmetry', 'PC2_asymmetry': different attempts for asymmetry calculation (all bad)

'xP_weight', 'xP_scale','xP_angle','xP_xy':
'xN_weight','xN_scale','xN_angle','xN_xy',
'yP_weight','yP_scale','yP_angle','yP_xy',
'yN_weight','yN_scale','yN_angle','yN_xy',
'PCA_angle':
'PCA_xy':
'PCA_shift_centroid_y', 'PCA_shift_centroid_x': position of the 2D_image centroid after shifting according to PCA

'DGI': from the old script 
"""

#The pixel resolution has to be provided to calculate everything in um(n) 

'" Results\' columns\' description \n\'neuron\':  neuron name extracted from file name\n\'subtype\': neuron name extracted from file name\n\'entry_point\': entry point of the neuron, read from seperate txt file\n\n\'volume\': volume of the masked 3D image in um3\n\'3D_convex_volume\': volume of the convex-hull for the 3D image in um3\n\'net_area\': area of the object in the 2D image (the 2D is a max projection of the 3D image) in um2\n\'area\': area of the convex_hull surronding the 2D image (obtained from skimage.measure.regionprops) in um2 \n\n\'density_2D\': solidity value obtained from skimage.measure.regionprops on the 2D image\n\'density_3D\': pixel count of 3D image/pixel count of 3D convexhull \n\'object_perimeter\': object perimenter value obtained from skimage.measure.regionprops on the 2D image\n\'convex_perimeter\': perimeter of the surronding convex_hull\n\n\'AP_spread\': (max-min) of value pixels index in last (x) dimension of the 2D image in um\n\'DV_spread\': (max-min) 

In [3]:
from tqdm import tqdm
import os
import re
import tifffile as tif

import utils.analysis as analysis
import utils.datautils as datautils

import numpy as np
from skimage.measure import regionprops, marching_cubes, mesh_surface_area, perimeter, find_contours
from skimage.measure.fit import spatial
import pandas as pd
import math
from scipy.spatial import ConvexHull

import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib.patches as mpatches
# import seaborn as sns
%matplotlib inline

from timeit import default_timer as timer
from multiprocessing import cpu_count, Process, Pool

In [4]:
#  input and save paths
data_path = '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/'
save_path = '/home/tli_project/Desktop/Data/T4/neurons/all_px_output/'
subtype = 'C'
stab_limit = 1 # No. of timepoints for stable branches calculation. all pixels are included if stab_limit = 1 
start_age = 37
px_resolution = (0.4, 0.076, 0.076) #z,y,x resolution
px_resolution = np.array(px_resolution)

In [5]:
# getting list of folders in data_path, where each folder has files for one neuron
N_folders = [name for name in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, name))]
N_folders = [f for f in N_folders if '_P' in f]
N_folders = sorted(N_folders)
N_folders

['211104_P36_N1',
 '211104_P36_N3',
 '211107_P36_N2',
 '211113_P36_N1',
 '211114_P36_N3',
 '211121_P36.5_N3',
 '220209_P36_N1']

In [6]:
# reading erntry_points file and converting it to dictionary of neu:[z,y,x]
EP_file = '/home/tli_project/Desktop/Data/T4/neurons/TLI_stage1_entry_points'
with open(EP_file) as f:
    entry_points = f.readlines()
entry_points = [l for l in entry_points if ': ' in l]
entry_points = [l.rstrip('\n').split(':') for l in entry_points]
entry_points = {l[0]: l[1].split(',') for l in entry_points}
entry_points = {neu:[int(x) for x in val] for neu,val in entry_points.items()}

In [7]:
results_cols = ['timepoint',
                'volume','3D_convex_volume', 'net_area', 'area',
                'AP_spread', 'DV_spread', 'eccentricity', 'ellipticity','PCA_elongation', 
                'circularity', 'feret_diameter_max', 
                'density_2D', 'density_3D', 
                'object_perimeter', 'convex_perimeter', 
                'compactness', 'surface_area', '3D_convex_surface',
                'convexity_2D', 'convexity_3D', 'orientation',
                'centroidZ','centroidY', 'centroidX', 'px_variance', 'px_std',
                'cen_EP_dis', 'cen_EP_angle', 'cen_EP_y', 'cen_EP_x',
                'cen_EP_norm_y', 'cen_EP_norm_x',
                'asymmetry', 'x_asymmetry', 'y_asymmetry', 'PC1_asymmetry', 'PC2_asymmetry',
                'xP_weight','xP_scale','xP_angle','xP_xy', #all of this part is for the results from PCA_analysis
                'xN_weight','xN_scale','xN_angle','xN_xy',
                'yP_weight','yP_scale','yP_angle','yP_xy',
                'yN_weight','yN_scale','yN_angle','yN_xy',
                'PC1_angle','PC1_x','PC1_y', 'DGI', 'DGI_angle',
                'PC1_angle2', 'PC1_cen_x', 'PC1_cen_y', 'PC1_ori_cen_x', 'PC1_ori_cen_y',
                'trans_volume', 'avg_trans_dist', 'max_trans_dist',
                'trans_cenPast_angle', 'trans_cen_angle', 
                'trans_cen_dis', 'trans_cen_y', 'trans_cen_x', 
                'trans_PCA_angle',  'AllPx_PCA_angle',
                'trans_EP_y','trans_EP_x', 'trans_EP_dis',
                'trans_EP_norm_y','trans_EP_norm_x',
                'stable_Allcen_y','stable_Allcen_x', 'stable_Allcen_dis',
                'trans_Allcen_y','trans_Allcen_x', 'trans_Allcen_dis',
                'PC1_shift_centroid_y', 'PC1_shift_centroid_x']

In [8]:
def get_theta_angle(img_2D, entry_point):
    theta = regionprops(img_2D)[0].orientation
    rotation_mat = np.matrix([[np.cos(theta), -np.sin(theta)],
                            [np.sin(theta), np.cos(theta)]])
    inv_rot_mat = np.linalg.inv(rotation_mat)


    coords = (np.argwhere(img_2D) - entry_point).T
    coords = np.vstack((coords[1],coords[0]))
    # rotate the original coordinates
    rotated_coords = rotation_mat * coords

    x_transformed, y_transformed = rotated_coords.A
    x_pos = x_transformed[x_transformed>=0]
    x_neg = x_transformed[x_transformed<0]
    y_pos = y_transformed[y_transformed>=0]
    y_neg = y_transformed[y_transformed<0]

    PCAs = {'x_pos':x_pos, 'x_neg':x_neg, 'y_pos':y_pos, 'y_neg':y_neg}
    PCs_fractions = {key:len(x) for key,x in PCAs.items()}
    
    PC1 = {max(PCs_fractions, key=PCs_fractions.get): PCAs[max(PCs_fractions, key=PCs_fractions.get)]}

    angular_rotation = {'x_pos':0, 'x_neg':180, 'y_pos':90, 'y_neg':270}
    element = next(iter(PC1))
    PC1_angle = np.rad2deg(theta) + angular_rotation[element]
    PC1_angle = np.deg2rad(PC1_angle)

    element = next(iter(PC1))
    if 'x_pos' in element:
        PC1_xys = rotated_coords[:,x_transformed>=0]
        PC1_ori_xys = coords[:,x_transformed>=0]
    elif 'x_neg' in element:
        PC1_xys = rotated_coords[:,x_transformed<0]
        PC1_ori_xys = coords[:,x_transformed<0]
    elif 'y_pos' in element:
        PC1_xys = rotated_coords[:,y_transformed>=0]
        PC1_ori_xys = coords[:,y_transformed>=0]
    elif 'y_neg' in element:
        PC1_xys = rotated_coords[:,y_transformed<0]
        PC1_ori_xys = coords[:,y_transformed<0]
    PC1_cen = PC1_xys.mean(axis=1)
    PC1_ori_cen = PC1_ori_xys.mean(axis=1)

    return PC1_angle, PC1_cen, PC1_ori_cen

In [9]:
def process_image(n_file):
    neuron_results = pd.DataFrame(columns=results_cols)
    entry_point = np.array(entry_points[n_file][1:]) # remove the first T element
    start_im = re.search('P(.*)_N', n_file)
    start_im = float(start_im.group(1))
    start_ana = int((start_age - start_im)/0.25)
    files_list = datautils.get_file_names(data_path+n_file, group_by='', 
                                        order=True, nested_files=False, 
                                        criteria='')
    neu_path = [f for f in files_list if 'clahe' in f.lower()][0]
    neuron = tif.imread(neu_path)
    mask_path = [f for f in files_list if 'mask' in f.lower()][0]
    mask = tif.imread(mask_path)
    neuron[mask==0] = 0
    neuron = neuron[start_ana:] #remove timepoints before start_ana from the 4D image array
    if stab_limit > 1:
        lifetimes = analysis.cal_lifetimes(neuron, save=False)
        neuron = lifetimes.copy()
        neuron[neuron<stab_limit] = 0
        trans_px = lifetimes.copy()
        trans_px[trans_px>=stab_limit] = 0
        trans_px[trans_px != 0] = 1
        neuron_results['trans_volume'] = trans_px.sum(axis=tuple(np.arange(len(trans_px.shape))[1:]))*px_resolution.prod()
    else:
        neuron_results['trans_volume'] = [0 for i in range(len(neuron))]
    neuron[neuron != 0] = 1
    neu_2D = neuron.max(axis=1)
    neuron_results['timepoint'] = [start_age+i*0.25 for i in range(len(neu_2D))]
    neuron_results['volume'] = neuron.sum(axis=tuple(np.arange(len(neuron.shape))[1:]))*px_resolution.prod()
    neuron_results['net_area'] = neu_2D.sum(axis=tuple(np.arange(len(neu_2D.shape))[1:]))*px_resolution[1:].prod()
    for t, stack in enumerate(neuron[stab_limit-1:], stab_limit-1):
        reg_prop_2D = regionprops(neu_2D[t])[0]
        reg_prop_3D = regionprops(stack)[0]
        neuron_results.loc[t,'density_2D'] = reg_prop_2D.solidity
        neuron_results.loc[t,'area'] = reg_prop_2D.area_convex*px_resolution[1:].prod()
        neuron_results.loc[t,'orientation'] = reg_prop_2D.orientation * (180/np.pi)
        neuron_results.loc[t,'eccentricity'] = reg_prop_2D.eccentricity
        neuron_results.loc[t,'ellipticity'] = (reg_prop_2D.axis_major_length - reg_prop_2D.axis_minor_length)/reg_prop_2D.axis_major_length
        neuron_results.loc[t,'feret_diameter_max'] = reg_prop_2D.feret_diameter_max*px_resolution[-1]
        neuron_results.loc[t,'convex_perimeter'] = perimeter(reg_prop_2D.image_convex)*px_resolution[-1]
        neuron_results.loc[t,'object_perimeter'] = reg_prop_2D.perimeter*px_resolution[-1]
        neuron_results.loc[t,'3D_convex_volume'] = reg_prop_3D.area_convex*px_resolution.prod()
        neuron_results.loc[t,'density_3D'] = neuron_results.loc[t,'volume']/neuron_results.loc[t,'3D_convex_volume']
        img_PC = np.argwhere(stack)
        neuron_results.loc[t,'DV_spread'] = (img_PC[:,1].max() - img_PC[:,1].min())*px_resolution[-1]
        neuron_results.loc[t,'AP_spread'] = (img_PC[:,2].max() - img_PC[:,2].min())*px_resolution[-1]

        centroid = np.array(reg_prop_3D.centroid)
        neuron_results.loc[t,['centroidZ','centroidY','centroidX']] = centroid
        p1 = centroid[1:] - entry_point[1:]
        p1[0] = -1*p1[0]
        neuron_results.loc[t,'cen_EP_dis'] = np.linalg.norm(p1)*px_resolution[-1]
        neuron_results.loc[t,['cen_EP_y','cen_EP_x']] = p1
        neuron_results.loc[t,'cen_EP_angle'] = np.rad2deg(np.arctan2(*p1) % (2 * np.pi))

        p1[1] = 6*p1[1]/4
        neuron_results.loc[t, ['cen_EP_norm_y', 'cen_EP_norm_x']] = p1[0], p1[1]

        verts, faces, normals, values = marching_cubes(stack, step_size=2, spacing=(px_resolution))
        neuron_results.loc[t,'surface_area'] = mesh_surface_area(verts, faces)
        convex_img_3D = reg_prop_3D.image_convex.astype('uint8')
        verts, faces, normals, values = marching_cubes(convex_img_3D, step_size=2, spacing=(px_resolution))
        neuron_results.loc[t,'3D_convex_surface'] = mesh_surface_area(verts, faces)
        neuron_results.loc[t,'convexity_3D'] = neuron_results.loc[t,'3D_convex_surface']/neuron_results.loc[t,'surface_area']

        asymmetry_values, PCA_df, shifted_coor, evals = analysis.metric_dump(neu_2D[t],entry_point[1:],plot=False)
        asymmetries = ['asymmetry', 'x_asymmetry', 'y_asymmetry', 'PC1_asymmetry', 'PC2_asymmetry']
        neuron_results.loc[t,asymmetries]  = asymmetry_values
        neuron_results.loc[t,'PCA_elongation'] = abs(evals[0]-evals[1])/evals.max()

        PCA_res = ['xP_weight','xP_scale','xP_angle','xP_xy',
                    'xN_weight','xN_scale','xN_angle','xN_xy',
                    'yP_weight','yP_scale','yP_angle','yP_xy',
                    'yN_weight','yN_scale','yN_angle','yN_xy']
        PCA_arr = np.array(PCA_df.loc[:, PCA_df.columns != 'axis'])
        neuron_results.loc[t,PCA_res]  = PCA_arr.ravel()
        
        neuron_results.loc[t,'PC1_angle'] = PCA_df.loc[PCA_df.Fraction_weight.idxmax()].Angle*-1

        neuron_results.loc[t,'PC1_x'] = PCA_df.loc[PCA_df.Fraction_weight.idxmax()].xy[0]
        neuron_results.loc[t,'PC1_y'] = PCA_df.loc[PCA_df.Fraction_weight.idxmax()].xy[1]

        # neuron_results.loc[t,'PC1_shift_centroid'] = (sum(shifted_coor[:,1]) / len(shifted_coor), sum(shifted_coor[:,0]) / len(shifted_coor))
        
        neuron_results.loc[t,'DGI'], neuron_results.loc[t,'DGI_angle'] = analysis.DGI_3D(stack,entry_point)

        neuron_results.loc[t,'px_variance'] = np.var(np.linalg.norm(img_PC, axis=0))/np.mean(np.linalg.norm(img_PC, axis=0))
        neuron_results.loc[t,'px_std'] = np.std(img_PC)
        
        neuron_results.loc[t,'PC1_angle2'], PC1_center, PC1_ori_cen = get_theta_angle(neu_2D[t],entry_point[1:])
        neuron_results.loc[t,'PC1_cen_x'], neuron_results.loc[t,'PC1_cen_y'] = PC1_center[0][0], PC1_center[1][0]
        neuron_results.loc[t,'PC1_ori_cen_x'], neuron_results.loc[t,'PC1_ori_cen_y'] = PC1_ori_cen

        if stab_limit > 1:
            all_centroid = np.array(regionprops(lifetimes[t])[0].centroid)
            All_px_2D = lifetimes[t].max(axis=0)
            All_px_2D[All_px_2D!=0] = 1
            asymmetry_values, PCA_df, shifted_coor, evals = analysis.metric_dump(All_px_2D,entry_point[1:],plot=False)
            neuron_results.loc[t,'AllPx_PCA_angle'] = PCA_df.loc[PCA_df.Fraction_weight.idxmax()].Angle*-1

            trans_2D = trans_px[t].max(axis=0)
            asymmetry_values, PCA_df, shifted_coor, evals = analysis.metric_dump(trans_2D,entry_point[1:],plot=False)
            neuron_results.loc[t,'trans_PCA_angle'] = PCA_df.loc[PCA_df.Fraction_weight.idxmax()].Angle*-1
            trans_centroid = np.array(regionprops(trans_px[t])[0].centroid)
            p1 = trans_centroid[1:] - centroid[1:]
            p1[0] = -1*p1[0]
            neuron_results.loc[t,'trans_cen_dis'] = np.linalg.norm(p1)*px_resolution[-1]
            neuron_results.loc[t,['trans_cen_y','trans_cen_x']] = p1
            neuron_results.loc[t,'trans_cen_angle'] = np.rad2deg(np.arctan2(*p1) % (2 * np.pi))

            p1 = trans_centroid[1:] - all_centroid[1:]
            p1[0] = -1*p1[0]
            neuron_results.loc[t,'trans_Allcen_dis'] = np.linalg.norm(p1)*px_resolution[-1]
            neuron_results.loc[t,['trans_Allcen_y','trans_Allcen_x']] = p1

            p1 = centroid[1:] - all_centroid[1:]
            p1[0] = -1*p1[0]
            neuron_results.loc[t,'stable_Allcen_dis'] = np.linalg.norm(p1)*px_resolution[-1]
            neuron_results.loc[t,['stable_Allcen_y','stable_Allcen_x']] = p1

            p1 = trans_centroid[1:] - entry_point[1:]
            p1[0] = -1*p1[0]
            neuron_results.loc[t,'trans_EP_dis'] = np.linalg.norm(p1)*px_resolution[-1]
            neuron_results.loc[t,['trans_EP_y','trans_EP_x']] = p1

            p1[1] = 6*p1[1]/4
            neuron_results.loc[t,['trans_EP_norm_y','trans_EP_norm_x']] = p1

            try:
                centroid = np.array(regionprops(neuron[t-1])[0].centroid)
                p1 = trans_centroid[1:] - centroid[1:]
                p1[0] = -1*p1[0]
                neuron_results.loc[t,'trans_cenPast_angle'] = np.rad2deg(np.arctan2(*p1) % (2 * np.pi))
            except:
                neuron_results.loc[t,'trans_cenPast_angle'] = 0

            contour = find_contours(neu_2D[t], 0.8)[0]
            contour_tx = find_contours(trans_2D, 0.8)
            results = []
            for test in contour_tx:
                if len(test)> 50:
                    for px in test:
                        dist, ind = spatial.KDTree(contour).query(px)
                        results.append(dist)  
            neuron_results.loc[t,'max_trans_dist'] = np.max(results)*px_resolution[-1]
            neuron_results.loc[t,'avg_trans_dist'] = np.mean(results)*px_resolution[-1]

    neuron_results['convexity_2D'] = neuron_results['convex_perimeter']/neuron_results['object_perimeter']
    neuron_results['circularity'] = (4*np.pi*neuron_results['area'])/(neuron_results['convex_perimeter']**2)
    neuron_results['compactness'] = (4*np.pi*neuron_results['net_area']) / neuron_results['object_perimeter']**2

    csv_name = save_path+n_file+'_stable'+str(stab_limit)+'_s'+subtype+'.csv'
    neuron_results.to_csv(csv_name)
    return neuron_results, 

for n_file in tqdm(N_folders):
    process_image(n_file)

  0%|          | 0/7 [00:00<?, ?it/s]

first 5 files
['/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211104_P36_N1/211104_P36_N1_R260_clahe.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211104_P36_N1/211104_P36_N1_R260_JE_mask2.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211104_P36_N1/211104_P36_N1_R260_JE_mask.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211104_P36_N1/211104_P36_N1_R260_ColRois.zip', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211104_P36_N1/211104_P36_N1_R260_5D.tif']


 14%|█▍        | 1/7 [03:18<19:50, 198.45s/it]

first 5 files
['/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211104_P36_N3/211104_P36_N3_R290_colRois.zip', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211104_P36_N3/211104_P36_N3_R290_clahe.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211104_P36_N3/211104_P36_N3_R290_JE_mask.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211104_P36_N3/211104_P36_N3_R290_5D.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211104_P36_N3/211104_P36_N3_CropRoi.roi']


 29%|██▊       | 2/7 [05:30<13:17, 159.52s/it]

...

 71%|███████▏  | 5/7 [09:51<03:34, 107.48s/it]

first 5 files
['/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211121_P36.5_N3/211121_P36.5_N3_R290_clahe.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211121_P36.5_N3/211121_P36.5_N3_R290_JE_mask.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211121_P36.5_N3/211121_P36.5_N3_R290_ColRois.zip', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211121_P36.5_N3/211121_P36.5_N3_R290_5D.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/211121_P36.5_N3/211121_P36.5_N3_CropRoi.roi']


 86%|████████▌ | 6/7 [12:17<02:00, 120.51s/it]

first 5 files
['/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/220209_P36_N1/220209_P36_N1_FlipV_R270_clahe.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/220209_P36_N1/220209_P36_N1_FlipV_R270_JE_mask.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/220209_P36_N1/220209_P36_N1_FlipV_R270_ColRois.zip', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/220209_P36_N1/220209_P36_N1_FlipV_R270_5D.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeC/220209_P36_N1/220209_P36_N1_CropRoi.roi']


100%|██████████| 7/7 [13:58<00:00, 119.76s/it]


In [10]:
# for i, n_file in tqdm(enumerate(N_folders)):
#     neuron_results = pd.DataFrame(columns=results_cols)
#     entry_point = entry_points[n_file][1:]
#     start_im = re.search('P(.*)_N', n_file)
#     start_im = float(start_im.group(1))
#     start_ana = int((start_age - start_im)/0.25)
#     files_list = datautils.get_file_names(data_path+n_file, group_by='', 
#                                         order=True, nested_files=False, 
#                                         criteria='')
#     neu_path = [f for f in files_list if 'clahe' in f.lower()][0]
#     neuron = tif.imread(neu_path)
#     mask_path = [f for f in files_list if 'mask' in f.lower()][0]
#     mask = tif.imread(mask_path)
#     neuron[mask==0] = 0
#     neuron = neuron[start_ana:] #remove timepoints before start_ana from the 4D image array
#     neuron[neuron != 0] = 1
#     neu_2D = neuron.max(axis=1)
#     neu_2D = neuron.max(axis=1)
#     neuron_results['timepoint'] = [start_age+i*0.25 for i in range(len(neu_2D))]
#     neuron_results['volume'] = neuron.sum(axis=tuple(np.arange(len(neuron.shape))[1:]))
#     neuron_results['net_area'] = neu_2D.sum(axis=tuple(np.arange(len(neu_2D.shape))[1:]))

#     reg_prop_2D = []
#     reg_prop_3D = []
#     for t, stack in enumerate(neuron):
#         reg_prop_2D.append(regionprops(neu_2D[t])[0])
#         reg_prop_3D.append(regionprops(stack)[0])
#         img_PC = reg_prop_3D[t].coords
#         neuron_results.loc[t,'DV_spread'] = (img_PC[:,1].max() - img_PC[:,1].min())*0.076
#         neuron_results.loc[t,'AP_spread'] = (img_PC[:,2].max() - img_PC[:,2].min())*0.076
#         neuron_results.loc[t,'density_3D'] = neuron_results.loc[t,'volume']/regionprops(stack)[0].area_convex

#         verts, faces, normals, values = marching_cubes(stack, step_size=2, spacing=(0.4,0.076,0.076))
#         neuron_results.loc[t,'surface_area'] = mesh_surface_area(verts, faces)
#         convex_img_3D = reg_prop_3D[t].image_convex.astype('uint8')
#         verts, faces, normals, values = marching_cubes(convex_img_3D, step_size=2, spacing=(0.4,0.076,0.076))
#         neuron_results.loc[t,'convexity_3D'] = mesh_surface_area(verts, faces)/neuron_results.loc[t,'surface_area']

#         asymmetry_values, PCA_df, shifted_coor = analysis.metric_dump(neu_2D[t],entry_point[1:],plot=False)
#         asymmetries = ['asymmetry', 'x_asymmetry', 'y_asymmetry', 'PC1_asymmetry', 'PC2_asymmetry']
#         neuron_results.loc[t,asymmetries]  = asymmetry_values

#         PCA_res = ['xP_weight','xP_scale','xP_angle','xP_xy',
#                     'xN_weight','xN_scale','xN_angle','xN_xy',
#                     'yP_weight','yP_scale','yP_angle','yP_xy',
#                     'yN_weight','yN_scale','yN_angle','yN_xy']
#         PCA_arr = np.array(PCA_df.loc[:, PCA_df.columns != 'axis'])
#         neuron_results.loc[t,PCA_res]  = PCA_arr.ravel()
#         neuron_results.loc[t,'PC1_angle'] = PCA_df.loc[PCA_df.Fraction_weight.idxmax()].Angle
#         neuron_results.loc[t,'PC1_x'] = PCA_df.loc[PCA_df.Fraction_weight.idxmax()].xy[0]
#         neuron_results.loc[t,'PC1_y'] = PCA_df.loc[PCA_df.Fraction_weight.idxmax()].xy[1]

#         neuron_results.loc[t,['centroidZ','centroidY','centroidX']] = reg_prop_3D[t].centroid
#         centroid_EP = np.array(entry_point)-np.array(reg_prop_3D[t].centroid)
#         neuron_results.loc[t,'cen_EP_dis'] = np.linalg.norm(centroid_EP[1:])
#         neuron_results.loc[t,['cen_EP_Y','cen_EP_X']] = centroid_EP[1:]
#         p1 = (centroid_EP[1],centroid_EP[2])
#         neuron_results.loc[t,'cen_EP_angle'] = np.rad2deg((np.arctan2(*p1[::-1])) % (2 * np.pi))


#     del neuron, neu_2D
#     neuron_results['density_2D'] = [r.solidity for r in reg_prop_2D]
#     # neuron_results['density_3D'] = [r.area_convex for r in reg_prop_3D]
#     # neuron_results['density_3D'] = neuron_results.loc[t,'volume']/neuron_results['density_3D']
#     neuron_results['area'] = [r.area_convex for r in reg_prop_2D]
#     neuron_results['orientation'] = [r.orientation for r in reg_prop_2D]
#     neuron_results['eccentricity'] = [r.eccentricity for r in reg_prop_2D]
#     neuron_results['feret_diameter_max'] = [r.feret_diameter_max for r in reg_prop_2D]
#     neuron_results['convex_perimeter'] = [perimeter(r.image_convex) for r in reg_prop_2D]
#     neuron_results['object_perimeter'] = [r.perimeter for r in reg_prop_2D]
#     neuron_results['convexity_2D'] = neuron_results['convex_perimeter']/neuron_results['object_perimeter']
#     neuron_results['circularity'] = (4*np.pi*neuron_results['area'])/(neuron_results['convex_perimeter']**2)
#     neuron_results['compactness'] = neuron_results['object_perimeter']**2/(4*np.pi*neuron_results['net_area'])
#     del reg_prop_2D, reg_prop_3D

#     csv_name = save_path+n_file+'_s'+subtype+'_new_111.csv'
#     neuron_results.to_csv(csv_name)

In [ ]:
print('memory cleared, and kernel stopped')
print('Job is DONE!')
os._exit(00)

: 

: 